Get a list of Kegg orthologies related to search terms relating to siderophores



## Setup

In [218]:
import pandas as pd
from collections import * 
import os 
import re
import json
from time import time
import numpy as np
import tarfile
import gzip
import itertools

In [2]:
workdir = '/scratch/bgrodner'
os.chdir(workdir)


'/scratch/bgrodner'

In [120]:
os.getcwd()

'/scratch/bgrodner'

In [121]:
os.listdir()

['siderophore_kofam.ipynb',
 '.DS_Store',
 '._.DS_Store',
 'ko00001.json',
 'siderophore_ko_contigs',
 'containers',
 'gradients_pfam_totals.ipynb',
 '._ko00001.json',
 'siderophore_kofam.txt']

## Get database searcher

Copied from https://www.biostars.org/p/492324/

Get dataframe

In [4]:
database = list()
for _, v in pd.read_json("ko00001.json").iterrows():
    d = v["children"]
    cat_1 = d["name"]
    for child_1 in d["children"]:
        cat_2 = child_1["name"] # Module?
        for child_2 in child_1["children"]:
            cat_3 = child_2["name"]
            if "children" in child_2:
                for child_3 in child_2["children"]:
                    cat_4 = child_3["name"]
                    fields = [cat_1, cat_2, cat_3, cat_4]
                    database.append(fields)
df_kegg = pd.DataFrame(database, columns=["Level_A", "Level_B", "Level_C", "Level_D"])
df_kegg.shape


(61993, 4)

In [7]:
df_kegg[:5]

,Level_A,Level_B,Level_C,Level_D,Level_D-KOs
0,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K00844 HK; hexokinase [EC:2.7.1.1],{K00844}
1,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K12407 GCK; glucokinase [EC:2.7.1.2],{K12407}
2,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K00845 glk; glucokinase [EC:2.7.1.2],{K00845}
3,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],K25026 glk; glucokinase [EC:2.7.1.2],{K25026}
4,09100 Metabolism,09101 Carbohydrate metabolism,00010 Glycolysis / Gluconeogenesis [PATH:ko00010],"K01810 GPI, pgi; glucose-6-phosphate isomeras...",{K01810}


### KO search

Build KO dictionary

In [12]:
ld = df_kegg['Level_D'].values
ld[:5]

array(['K00844  HK; hexokinase [EC:2.7.1.1]',
       'K12407  GCK; glucokinase [EC:2.7.1.2]',
       'K00845  glk; glucokinase [EC:2.7.1.2]',
       'K25026  glk; glucokinase [EC:2.7.1.2]',
       'K01810  GPI, pgi; glucose-6-phosphate isomerase [EC:5.3.1.9]'],
      dtype=object)

In [110]:
dict_ko_name = {}
for name in ld:
    ko = re.search(r"^\w+",name)[0]
    dict_ko_name[ko] = name

In [111]:
dict_ko_name['K00844']

'K00844  HK; hexokinase [EC:2.7.1.1]'

Build KO keyword search dictionary

In [132]:
[s for s in re.split(';|,|/|\s', 'K00844  HK; abcd/hexokinase [EC:2.7.1.1]') if s]

['K00844', 'HK', 'abcd', 'hexokinase', '[EC:2.7.1.1]']

In [133]:
dict_search_result = defaultdict(list)
for result in ld:
    r_split = [s for s in re.split(';|,|-|\(|\)|/|\s', result) if s]
    for search in r_split:
        dict_search_result[search].append(result)

Get siderophore KOs

In [134]:
sidero_results = dict_search_result['siderophore']
sidero_results

['K25286  fagD, cchF, irp1A, piaA; iron-siderophore transport system substrate-binding protein',
 'K23186  fepD, fagA, cchC, desH; iron-siderophore transport system permease protein',
 'K23187  fepG, fagB, cchD, desG; iron-siderophore transport system permease protein',
 'K23188  fepC, fagC, cchE, desF; iron-siderophore transport system ATP-binding protein [EC:7.2.2.17 7.2.2.-]',
 'K25286  fagD, cchF, irp1A, piaA; iron-siderophore transport system substrate-binding protein',
 'K23186  fepD, fagA, cchC, desH; iron-siderophore transport system permease protein',
 'K23187  fepG, fagB, cchD, desG; iron-siderophore transport system permease protein',
 'K23188  fepC, fagC, cchE, desF; iron-siderophore transport system ATP-binding protein [EC:7.2.2.17 7.2.2.-]',
 'K25109  sirA, fecB, cbrA; iron-siderophore transport system substrate-binding protein',
 'K25110  sirB, fecC, cbrB; iron-siderophore transport system permease protein',
 'K25111  sirC, fecD, cbrC; iron-siderophore transport system p

In [135]:
sidero_KOs = set([re.search(r"^\w+",r)[0] for r in sidero_results])
sidero_KOs

{'K08197',
 'K08225',
 'K16090',
 'K23186',
 'K23187',
 'K23188',
 'K25109',
 'K25110',
 'K25111',
 'K25130',
 'K25282',
 'K25283',
 'K25284',
 'K25285',
 'K25286'}

Enterobactin

In [136]:
enterobactin_results = dict_search_result['enterobactin']
enterobactin_results

['K02362  entD, pptT; enterobactin synthetase component D / holo-[acyl-carrier protein] synthase [EC:6.3.2.14 2.7.8.7]',
 'K02362  entD, pptT; enterobactin synthetase component D / holo-[acyl-carrier protein] synthase [EC:6.3.2.14 2.7.8.7]',
 'K23185  fepB; ferric enterobactin transport system substrate-binding protein',
 'K18895  iroC; ATP-binding cassette, subfamily B, salmochelin/enterobactin exporter',
 'K19611  fepA, pfeA, iroN, pirA; ferric enterobactin receptor',
 'K02362  entD, pptT; enterobactin synthetase component D / holo-[acyl-carrier protein] synthase [EC:6.3.2.14 2.7.8.7]',
 'K18895  iroC; ATP-binding cassette, subfamily B, salmochelin/enterobactin exporter',
 'K23185  fepB; ferric enterobactin transport system substrate-binding protein',
 'K08225  entS; MFS transporter, ENTS family, enterobactin (siderophore) exporter',
 'K19611  fepA, pfeA, iroN, pirA; ferric enterobactin receptor',
 'K23725  iroB; enterobactin C-glucosyltransferase [EC:2.4.1.369]',
 'K07214  fes; iron

In [137]:
enterobactin_KOs = set([re.search(r"^\w+",r)[0] for r in enterobactin_results])
enterobactin_KOs

{'K02362',
 'K07214',
 'K08225',
 'K18895',
 'K19611',
 'K23185',
 'K23722',
 'K23725'}

Iron transport

In [144]:
transport_results = dict_search_result['transport']
transporter_results = dict_search_result['transporter']
iron_results = dict_search_result['iron']
ferric_results = dict_search_result['ferric']

transport_KOs = set([re.search(r"^\w+",r)[0] for r in transport_results])
transporter_KOs = set([re.search(r"^\w+",r)[0] for r in transporter_results])
iron_KOs = set([re.search(r"^\w+",r)[0] for r in iron_results])
ferric_KOs = set([re.search(r"^\w+",r)[0] for r in ferric_results])

iron_transport_KOs = (    
    (transport_KOs & iron_KOs) | 
    (transporter_KOs & iron_KOs) | 
    (transport_KOs & ferric_KOs) | 
    (transporter_KOs & ferric_KOs)
)
iron_transport_KOs

{'K02010',
 'K02011',
 'K02012',
 'K02013',
 'K02015',
 'K02016',
 'K02068',
 'K02069',
 'K04758',
 'K04759',
 'K07243',
 'K07490',
 'K09818',
 'K09819',
 'K09820',
 'K10123',
 'K10829',
 'K11604',
 'K11605',
 'K11606',
 'K11607',
 'K11704',
 'K11705',
 'K11706',
 'K11707',
 'K11708',
 'K11709',
 'K11710',
 'K12346',
 'K14685',
 'K15113',
 'K19791',
 'K19792',
 'K22736',
 'K22737',
 'K23181',
 'K23182',
 'K23183',
 'K23184',
 'K23185',
 'K23186',
 'K23187',
 'K23188',
 'K23227',
 'K23228',
 'K25109',
 'K25110',
 'K25111',
 'K25130',
 'K25282',
 'K25283',
 'K25284',
 'K25285',
 'K25286',
 'K25287',
 'K25288',
 'K25289',
 'K25290',
 'K25308'}

Ferritin

In [ ]:
ferritin_results = dict_search_result['ferritin']
bacterioferritin_results = dict_search_result['bacterioferritin']

ferritin_KOs = set([re.search(r"^\w+",r)[0] for r in ferritin_results])
bacterioferritin_KOs = set([re.search(r"^\w+",r)[0] for r in bacterioferritin_results])

ferritin_KOs = ferritin_KOs | bacterioferritin_KOs

ferritin_results, bacterioferritin_results, ferritin_KOs

(['K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K18495  FTMT; ferritin, mitochondrial [EC:1.16.3.1]',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K18495  FTMT; ferritin, mitochondrial [EC:1.16.3.1]',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K02217  ftnA, ftn; ferritin [EC:1.16.3.2]',
  'K02255  ftnB; ferritin-like protein 2'],
 ['K03594  bfr; bacterioferritin [EC:1.16.3.1]',
  'K22336  bfrB; bacterioferritin B [EC:1.16.3.1]',
  'K02192  bfd; bacterioferritin-associated ferredoxin'],
 {'K00522',
  'K02192',
  'K02217',
  'K02255',
  'K03594',
  'K13625',
  'K18495',
  'K22336'})

ZIP family

In [143]:
zinc_results = dict_search_result['zinc']

zinc_KOs = set([re.search(r"^\w+",r)[0] for r in zinc_results])

zinc_transporter_KOs = (transporter_KOs & zinc_KOs)
zinc_transporter_KOs

{'K07238',
 'K14688',
 'K14689',
 'K14690',
 'K14691',
 'K14692',
 'K14693',
 'K14694',
 'K14695',
 'K14696',
 'K14697',
 'K14709',
 'K14710',
 'K14711',
 'K14712',
 'K14713',
 'K14714',
 'K14715',
 'K14716',
 'K14717',
 'K14718',
 'K14719',
 'K14720',
 'K16074',
 'K16267'}

Transferrin

In [146]:
transferrin_results = dict_search_result['transferrin']

transferrin_KOs = set([re.search(r"^\w+",r)[0] for r in transferrin_results])
transferrin_results, transferrin_KOs

(['K06503  TFRC, CD71; transferrin receptor',
  'K14736  TF; transferrin',
  'K23910  TFR2; transferrin receptor protein 2',
  'K06503  TFRC, CD71; transferrin receptor',
  'K14736  TF; transferrin',
  'K06503  TFRC, CD71; transferrin receptor',
  'K06503  TFRC, CD71; transferrin receptor',
  'K06503  TFRC, CD71; transferrin receptor',
  'K14736  TF; transferrin',
  'K06503  TFRC, CD71; transferrin receptor',
  'K06503  TFRC, CD71; transferrin receptor',
  'K14736  TF; transferrin',
  'K06503  TFRC, CD71; transferrin receptor',
  'K23910  TFR2; transferrin receptor protein 2',
  'K16087  TC.FEV.OM3, tbpA, hemR, lbpA, hpuB, bhuR, hugA, hmbR; hemoglobin/transferrin/lactoferrin receptor protein',
  'K06503  TFRC, CD71; transferrin receptor',
  'K14736  TF; transferrin',
  'K06503  TFRC, CD71; transferrin receptor'],
 {'K06503', 'K14736', 'K16087', 'K23910'})

Ferritin

In [149]:
ferritin_results = dict_search_result['ferritin']
bacterioferritin_results = dict_search_result['bacterioferritin']

ferritin_KOs = set([re.search(r"^\w+",r)[0] for r in ferritin_results])
bacterioferritin_KOs = set([re.search(r"^\w+",r)[0] for r in bacterioferritin_results])

ferritin_KOs = ferritin_KOs | bacterioferritin_KOs

ferritin_results, bacterioferritin_results, ferritin_KOs

(['K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K18495  FTMT; ferritin, mitochondrial [EC:1.16.3.1]',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K18495  FTMT; ferritin, mitochondrial [EC:1.16.3.1]',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K00522  FTH1; ferritin heavy chain [EC:1.16.3.1]',
  'K13625  FTL; ferritin light chain',
  'K02217  ftnA, ftn; ferritin [EC:1.16.3.2]',
  'K02255  ftnB; ferritin-like protein 2'],
 ['K03594  bfr; bacterioferritin [EC:1.16.3.1]',
  'K22336  bfrB; bacterioferritin B [EC:1.16.3.1]',
  'K02192  bfd; bacterioferritin-associated ferredoxin'],
 {'K00522',
  'K02192',
  'K02217',
  'K02255',
  'K03594',
  'K13625',
  'K18495',
  'K22336'})

Ferredoxin (NOTE: TO BE SEPARATED INTO FERREDOXIN USE AN FERREDOXIN REPLACEMENT WITH FLAVODOXIN)

In [ ]:
ferredoxin_results = dict_search_result['ferredoxin']
ferredoxin_KOs = set([re.search(r"^\w+",r)[0] for r in ferredoxin_results])

ferredoxin_KOs

{'K00169',
 'K00170',
 'K00171',
 'K00172',
 'K00174',
 'K00175',
 'K00176',
 'K00177',
 'K00179',
 'K00180',
 'K00186',
 'K00187',
 'K00188',
 'K00189',
 'K00204',
 'K00205',
 'K00284',
 'K00366',
 'K00367',
 'K00392',
 'K00528',
 'K00529',
 'K00532',
 'K00533',
 'K00534',
 'K02192',
 'K02572',
 'K02573',
 'K02574',
 'K02639',
 'K02641',
 'K03737',
 'K03855',
 'K03863',
 'K04090',
 'K04755',
 'K05337',
 'K05524',
 'K05710',
 'K06441',
 'K11107',
 'K11260',
 'K11389',
 'K14578',
 'K14581',
 'K14603',
 'K14750',
 'K15752',
 'K15753',
 'K15762',
 'K16304',
 'K17892',
 'K18087',
 'K18088',
 'K18089',
 'K18090',
 'K18227',
 'K18248',
 'K18249',
 'K18253',
 'K18254',
 'K21231',
 'K21480',
 'K21567',
 'K21834',
 'K22070',
 'K22071',
 'K22338',
 'K22339',
 'K22340',
 'K22341',
 'K22360',
 'K22362',
 'K23976',
 'K24244',
 'K24245',
 'K24530',
 'K26113',
 'K26114',
 'K26115'}

Flavodoxin

In [ ]:
flavodoxin_results = dict_search_result['flavodoxin']
flavodoxin_KOs = set([re.search(r"^\w+",r)[0] for r in flavodoxin_results])

flavodoxin_results, flavodoxin_KOs

(['K03737  por, nifJ; pyruvate-ferredoxin/flavodoxin oxidoreductase [EC:1.2.7.1 1.2.7.-]',
  'K03737  por, nifJ; pyruvate-ferredoxin/flavodoxin oxidoreductase [EC:1.2.7.1 1.2.7.-]',
  'K03737  por, nifJ; pyruvate-ferredoxin/flavodoxin oxidoreductase [EC:1.2.7.1 1.2.7.-]',
  'K03737  por, nifJ; pyruvate-ferredoxin/flavodoxin oxidoreductase [EC:1.2.7.1 1.2.7.-]',
  'K03737  por, nifJ; pyruvate-ferredoxin/flavodoxin oxidoreductase [EC:1.2.7.1 1.2.7.-]',
  'K00528  fpr; ferredoxin/flavodoxin---NADP+ reductase [EC:1.18.1.2 1.19.1.1]',
  'K21567  fnr; ferredoxin/flavodoxin---NADP+ reductase [EC:1.18.1.2 1.19.1.1]',
  'K03840  fldB; flavodoxin II',
  'K03839  fldA, nifF, isiB; flavodoxin I'],
 {'K00528', 'K03737', 'K03839', 'K03840', 'K21567'})

Photosystem

In [152]:
photosystem_results = dict_search_result['photosystem']
photosystem_KOs = set([re.search(r"^\w+",r)[0] for r in photosystem_results])

photosystem_KOs

{'K02689',
 'K02690',
 'K02691',
 'K02692',
 'K02693',
 'K02694',
 'K02695',
 'K02696',
 'K02697',
 'K02698',
 'K02699',
 'K02700',
 'K02701',
 'K02702',
 'K02703',
 'K02704',
 'K02705',
 'K02706',
 'K02707',
 'K02708',
 'K02709',
 'K02710',
 'K02711',
 'K02712',
 'K02713',
 'K02714',
 'K02716',
 'K02717',
 'K02718',
 'K02719',
 'K02720',
 'K02721',
 'K02722',
 'K02723',
 'K02724',
 'K03541',
 'K03542',
 'K08901',
 'K08902',
 'K08903',
 'K08904',
 'K08905',
 'K08940',
 'K08941',
 'K08942',
 'K08943',
 'K14332'}

Nitrate/nitrite

In [154]:
nitrate_results = dict_search_result['nitrate']
nitrite_results = dict_search_result['nitrite']
nitrate_KOs = set([re.search(r"^\w+",r)[0] for r in nitrate_results])
nitrite_KOs = set([re.search(r"^\w+",r)[0] for r in nitrite_results])

nitrate_nitrate_KOs = nitrate_KOs | nitrite_KOs

nitrate_nitrate_KOs

{'K00360',
 'K00361',
 'K00362',
 'K00363',
 'K00366',
 'K00367',
 'K00368',
 'K00370',
 'K00371',
 'K00372',
 'K00373',
 'K00374',
 'K02567',
 'K02568',
 'K02570',
 'K02571',
 'K02575',
 'K02598',
 'K03385',
 'K04017',
 'K04018',
 'K07673',
 'K07674',
 'K07684',
 'K07685',
 'K10534',
 'K10850',
 'K15576',
 'K15577',
 'K15578',
 'K15579',
 'K15864',
 'K15876',
 'K17877',
 'K19343',
 'K21563',
 'K21990',
 'K22962',
 'K26138',
 'K26139',
 'K26548',
 'K26549',
 'K26551',
 'K26552',
 'K26553',
 'K26554',
 'K26562',
 'K26563'}

Glutamate synthase

In [158]:
glutamate_results = dict_search_result['glutamate']
synthase_results = dict_search_result['synthase']

glutamate_KOs = set([re.search(r"^\w+",r)[0] for r in glutamate_results])
synthase_KOs = set([re.search(r"^\w+",r)[0] for r in synthase_results])

glutamate_synthase_KOs = glutamate_KOs & synthase_KOs

glutamate_synthase_KOs

{'K00264', 'K00265', 'K00266', 'K00284', 'K09681', 'K21949', 'K24034'}

Carbonic anhydrase

In [159]:
carbonic_results = dict_search_result['carbonic']
anhydrase_results = dict_search_result['anhydrase']

carbonic_KOs = set([re.search(r"^\w+",r)[0] for r in carbonic_results])
anhydrase_KOs = set([re.search(r"^\w+",r)[0] for r in anhydrase_results])

carbonic_anhydrase_KOs = carbonic_KOs & anhydrase_KOs

carbonic_anhydrase_KOs

{'K01672', 'K01673', 'K01674', 'K01726', 'K01743', 'K18245', 'K18246'}

bicarbonate

In [160]:
bicarbonate_results = dict_search_result['bicarbonate']
bicarbonate_KOs = set([re.search(r"^\w+",r)[0] for r in bicarbonate_results])

bicarbonate_KOs

{'K11950',
 'K11951',
 'K11952',
 'K11953',
 'K13575',
 'K13857',
 'K13858',
 'K13859',
 'K13860',
 'K13861'}

fructose bisphosphatase

In [161]:
fructose_results = dict_search_result['fructose']
bisphosphatase_results = dict_search_result['bisphosphatase']

fructose_KOs = set([re.search(r"^\w+",r)[0] for r in fructose_results])
bisphosphatase_KOs = set([re.search(r"^\w+",r)[0] for r in bisphosphatase_results])

fructose_bisphosphatase_KOs = fructose_KOs & bisphosphatase_KOs

fructose_bisphosphatase_KOs

{'K01086', 'K02446', 'K03841', 'K04041', 'K11532', 'K14634'}

fructose bisphosphatase

Manual addition

In [156]:
manual_results = [
    'K13255 fhuF; ferric iron reductase protein FhuF',
    'K16088 TC.FEV.OM1, fhuE, fpvA, fptA; outer-membrane receptor for ferric coprogen and ferric-rhodotorulic acid',
    'K23721 iroE; apo-salmochelin esterase [EC:3.1.1.107]',
    'K23723 iroD; iron(III)-salmochelin esterase [EC:3.1.1.109]',
    'K03832 tonB; periplasmic protein TonB',
    'K05368 fre, ubiB; NAD(P)H-flavin reductase [EC:1.5.1.41]',
    'K07230 p19, ftrA; periplasmic iron binding protein',    
    'K12347 SLC11A1, NRAMP1; natural resistance-associated macrophage protein 1',
    'K21398 SLC11A2, DMT1, NRAMP2; natural resistance-associated macrophage protein 2',
    'K03322 mntH; manganese transport protein',
    'K21398 SLC11A2, DMT1, NRAMP2; natural resistance-associated macrophage protein 2',
    'K26562 NPF5, NRT1, NAET; MFS transporter, POT/PTR family, peptide/nitrate/nicotianamine transporter',
    'K22552 mmcO; multicopper oxidase [EC:1.16.3.1]',
    'K22350 mcoA; manganese oxidase [EC:1.16.3.3]',
    'K12243 pchR; AraC family transcriptional regulator, transcriptional activator of the genes for pyochelin and ferripyochelin receptors',
    'K05374 irp4, ybtT; yersiniabactin synthetase, thioesterase component',
    'K15721 fyuA; pesticin/yersiniabactin receptor',
    'K08906 petJ; cytochrome c6',
    'K02638 petE; plastocyanin',
    'K03320 amt, AMT, MEP; ammonium transporter, Amt family',
    'K11959 urtA; urea transport system substrate-binding protein',
    'K01595 ppc; phosphoenolpyruvate carboxylase [EC:4.1.1.31]',
    'K00592 RBCMT; [ribulose-bisphosphate carboxylase]/[fructose-bisphosphate aldolase]-lysine N-methyltransferase [EC:2.1.1.127 2.1.1.259]',
    'K01601 rbcL, cbbL; ribulose-bisphosphate carboxylase large chain [EC:4.1.1.39]',
    'K01602 rbcS, cbbS; ribulose-bisphosphate carboxylase small chain [EC:4.1.1.39]',
    'K27935 rbcX; RuBisCO chaperone RbcX',
    'K00927 PGK, pgk; phosphoglycerate kinase [EC:2.7.2.3]',
    'K05715 2PGK; 2-phosphoglycerate kinase [EC:2.7.2.16]',
    'K01803 TPI, tpiA; triosephosphate isomerase (TIM) [EC:5.3.1.1]',
    'K25224 gapdh; glyceraldehyde-3-phosphate dehydrogenase (arsenate-transferring) [EC:1.2.1.107]',
    'K00134 GAPDH, gapA; glyceraldehyde 3-phosphate dehydrogenase (phosphorylating) [EC:1.2.1.12]',
    'K00615 E2.2.1.1, tktA, tktB; transketolase [EC:2.2.1.1]',
    'K01783 rpe, RPE; ribulose-phosphate 3-epimerase [EC:5.1.3.1]',
    'K00855 PRK, prkB; phosphoribulokinase [EC:2.7.1.19]',
    'K01623 ALDO; fructose-bisphosphate aldolase, class I [EC:4.1.2.13]',
    'K01624 FBA, fbaA; fructose-bisphosphate aldolase, class II [EC:4.1.2.13]',
    'K11645 fbaB; fructose-bisphosphate aldolase, class I [EC:4.1.2.13]',
    'K03699 tlyC; magnesium and cobalt exporter, CNNM family',
    'K16302 CNNM; metal transporter CNNM',
    'K00518 sodN; nickel superoxide dismutase [EC:1.15.1.1]',
    'K04564 SOD2; superoxide dismutase, Fe-Mn family [EC:1.15.1.1]',
    'K04565 SOD1; superoxide dismutase, Cu-Zn family [EC:1.15.1.1]',
    'K04569 CCS; copper chaperone for superoxide dismutase',
    'K16627 SOD3; superoxide dismutase, Cu-Zn family [EC:1.15.1.1]',
    'K00909 GRK1_7; rhodopsin kinase [EC:2.7.11.14]',
    'K04250 RHO, OPN2; rhodopsin',
    'K04641 bop; bacteriorhodopsin',
    'K04642 hop; halorhodopsin',
    'K23306 brz; bacteriorhodopsin-regulating zinc finger protein',
    'K01012 bioB; biotin synthase [EC:2.8.1.6]',
    'K18609 pldh; pyridoxal 4-dehydrogenase [EC:1.1.1.107]',
    'K00941 thiD; hydroxymethylpyrimidine/phosphomethylpyrimidine kinase [EC:2.7.1.49 2.7.4.7]',
    'K03147 thiC; phosphomethylpyrimidine synthase [EC:4.1.99.17]',
]
manual_KOs = set([re.search(r"^\w+",r)[0] for r in manual_results])
manual_KOs

{'K00134',
 'K00518',
 'K00592',
 'K00615',
 'K00855',
 'K00909',
 'K00927',
 'K00941',
 'K01012',
 'K01595',
 'K01601',
 'K01602',
 'K01623',
 'K01624',
 'K01783',
 'K01803',
 'K02638',
 'K03147',
 'K03320',
 'K03322',
 'K03699',
 'K03832',
 'K04250',
 'K04564',
 'K04565',
 'K04569',
 'K04641',
 'K04642',
 'K05368',
 'K05374',
 'K05715',
 'K07230',
 'K08906',
 'K11645',
 'K11959',
 'K12243',
 'K12347',
 'K13255',
 'K15721',
 'K16088',
 'K16302',
 'K16627',
 'K18609',
 'K21398',
 'K22350',
 'K22552',
 'K23306',
 'K23721',
 'K23723',
 'K25224',
 'K26562',
 'K27935'}

### Pathway search

Build search dictionary

In [61]:
lc = df_kegg[['Level_C','Level_D']].values
lc[:5]

array([['00010 Glycolysis / Gluconeogenesis [PATH:ko00010]',
        'K00844  HK; hexokinase [EC:2.7.1.1]'],
       ['00010 Glycolysis / Gluconeogenesis [PATH:ko00010]',
        'K12407  GCK; glucokinase [EC:2.7.1.2]'],
       ['00010 Glycolysis / Gluconeogenesis [PATH:ko00010]',
        'K00845  glk; glucokinase [EC:2.7.1.2]'],
       ['00010 Glycolysis / Gluconeogenesis [PATH:ko00010]',
        'K25026  glk; glucokinase [EC:2.7.1.2]'],
       ['00010 Glycolysis / Gluconeogenesis [PATH:ko00010]',
        'K01810  GPI, pgi; glucose-6-phosphate isomerase [EC:5.3.1.9]']],
      dtype=object)

In [68]:
re.search(r"^\w+", '00010 Glycolysis / Gluconeogenesis [PATH:ko00010]')[0]

'00010'

In [71]:
dict_path_result = defaultdict(list)
for path_, result in lc:
    path = re.search(r"^\w+", path_)[0]
    dict_path_result[path].append(result)

Get siderophore synthesis KOs

In [70]:
sidero_path = '01053'

In [72]:
sidero_path_results = dict_path_result[sidero_path]
sidero_path_results

['K01851  pchA; salicylate biosynthesis isochorismate synthase [EC:5.4.4.2]',
 'K02552  menF; menaquinone-specific isochorismate synthase [EC:5.4.4.2]',
 'K02361  entC; isochorismate synthase [EC:5.4.4.2]',
 'K01252  entB, dhbB, vibB, mxcF; bifunctional isochorismate lyase / aryl carrier protein [EC:3.3.2.1 6.3.2.14]',
 'K00216  entA; 2,3-dihydro-2,3-dihydroxybenzoate dehydrogenase [EC:1.3.1.28]',
 'K02362  entD, pptT; enterobactin synthetase component D / holo-[acyl-carrier protein] synthase [EC:6.3.2.14 2.7.8.7]',
 'K02363  entE, dhbE, vibE, mxcE; 2,3-dihydroxybenzoate---[aryl-carrier protein] ligase [EC:6.3.2.14 6.2.1.71]',
 'K02364  entF; L-serine---[L-seryl-carrier protein] ligase [EC:6.3.2.14 6.2.1.72]',
 'K24147  entH; proofreading thioesterase EntH',
 'K04778  vibH; vibriobactin synthetase',
 'K12237  vibF; nonribosomal peptide synthetase VibF',
 'K04780  dhbF; glyine---[glycyl-carrier protein] ligase [EC:6.2.1.66]',
 'K15653  mxcG; nonribosomal peptide synthetase MxcG',
 'K156

In [73]:
sidero_path_KOs = set([re.search(r"^\w+",r)[0] for r in sidero_path_results])
sidero_path_KOs

{'K00216',
 'K01252',
 'K01851',
 'K02361',
 'K02362',
 'K02363',
 'K02364',
 'K02552',
 'K04778',
 'K04780',
 'K04781',
 'K04782',
 'K04783',
 'K04784',
 'K04785',
 'K04786',
 'K04787',
 'K04788',
 'K04789',
 'K04790',
 'K04791',
 'K04792',
 'K04793',
 'K12237',
 'K12238',
 'K12239',
 'K12240',
 'K12241',
 'K15652',
 'K15653',
 'K15681',
 'K24108',
 'K24109',
 'K24110',
 'K24111',
 'K24112',
 'K24147'}

### Merge KO lists

In [163]:
merged_KOs = (
    sidero_KOs | 
    enterobactin_KOs | 
    iron_transport_KOs | 
    ferritin_KOs | 
    zinc_transporter_KOs | 
    transferrin_KOs | 
    ferritin_KOs | 
    ferredoxin_KOs | 
    flavodoxin_KOs |
    photosystem_KOs |
    nitrate_nitrate_KOs | 
    glutamate_synthase_KOs | 
    carbonic_anhydrase_KOs | 
    bicarbonate_KOs | 
    fructose_bisphosphatase_KOs | 
    manual_KOs | 
    sidero_path_KOs
)
len(merged_KOs)

395

Write to file

In [250]:
with open('list_KOs.txt', 'w') as f:
    for line in list(merged_KOs):
        f.write(f"{line}\n")

## Search Gradients assemblies

### Test search

Get G1 filenames

In [74]:
kofam_dir = '/mnt/nfs/projects/armbrust-metat/gradients1/g1_station_ns_metat/assemblies/kofam'
kofam_fns = os.listdir(kofam_dir)
kofam_fns

['G1NS.S11C1_3um.kofam2021.domtblout.tab',
 'G1NS.S11C1_0_2um.kofam2021.csv',
 'G1NS.S09C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S06C1_3um.kofam2021.domtblout.tab',
 'G1NS.S11C1_3um.kofam2021.csv',
 'G1NS.S02C1_3um.kofam2021.csv',
 'G1NS.S08C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S10_0_2um.kofam2021.domtblout.tab',
 'G1NS.S10_0_2um.kofam2021.csv',
 'G1NS.S06C1_0_2um.kofam2021.csv',
 'G1NS.S06C1_3um.kofam2021.csv',
 'G1NS.S12C1_3um.kofam2021.csv',
 'G1NS.S08C1_3um.kofam2021.csv',
 'G1NS.S06C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S09C1_0_2um.kofam2021.csv',
 'G1NS.S04C1_3um.kofam2021.csv',
 'G1NS.S13C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S13C1_3um.kofam2021.domtblout.tab',
 'G1NS.S12C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S08C1_3um.kofam2021.domtblout.tab',
 'G1NS.S13C1_0_2um.kofam2021.csv',
 'G1NS.S04C1_3um.kofam2021.domtblout.tab',
 'G1NS.S02C1_3um.kofam2021.domtblout.tab',
 'G1NS.S11C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S04C1_0_2um.kofam2021.domtblout.tab',
 'G1NS.S08C1_

In [76]:
kofam_fns_csv = [f for f in kofam_fns if '.csv' in f]
kofam_fns_csv

['G1NS.S11C1_0_2um.kofam2021.csv',
 'G1NS.S11C1_3um.kofam2021.csv',
 'G1NS.S02C1_3um.kofam2021.csv',
 'G1NS.S10_0_2um.kofam2021.csv',
 'G1NS.S06C1_0_2um.kofam2021.csv',
 'G1NS.S06C1_3um.kofam2021.csv',
 'G1NS.S12C1_3um.kofam2021.csv',
 'G1NS.S08C1_3um.kofam2021.csv',
 'G1NS.S09C1_0_2um.kofam2021.csv',
 'G1NS.S04C1_3um.kofam2021.csv',
 'G1NS.S13C1_0_2um.kofam2021.csv',
 'G1NS.S08C1_0_2um.kofam2021.csv',
 'G1NS.S14C1_3um.kofam2021.csv',
 'G1NS_kofam2021_ALL_intermediate.csv',
 'G1NS.S09C1_3um.kofam2021.csv',
 'G1NS_kofam2021_ALL.csv',
 'G1NS.S13C1_3um.kofam2021.csv',
 'G1NS.S02C1_0_2um.kofam2021.csv',
 'G1NS.S04C1_0_2um.kofam2021.csv',
 'G1NS.S14C1_0_2um.kofam2021.csv',
 'G1NS.S12C1_0_2um.kofam2021.csv']

See if there are any siderophore transporter kos

In [239]:
fn = 'G1NS.S11C1_3um.kofam2021.csv'
fn_full = kofam_dir + '/' + fn

dict_transp_ko_contig = defaultdict(list)
dict_synth_ko_contig = defaultdict(list)
i = 0
times = []
with open(fn_full,'r') as f:
    for line in f:
        if i < 50000:
            t0 = time()
            l = line.split(',')
            l = list(filter(None, l))
            ko = l[4].replace('"','')
            if ko in merged_KOs:
                contig = l[1].replace('"','')
                dict_transp_ko_contig[ko].append(contig)
            t1 = time()
            times.append(t1-t0)
            i += 1
        else:
            break


np.mean(times)

1.6600322723388671e-06

In [247]:
(582792914 * 2.513885498046875e-06) / 60

24.41791091448466

In [ ]:
[[dict_ko_name[k], len(v)] for k, v in dict_transp_ko_contig.items()]

[['K00216  entA; 2,3-dihydro-2,3-dihydroxybenzoate dehydrogenase [EC:1.3.1.28]',
  1519],
 ['K01252  entB, dhbB, vibB, mxcF; bifunctional isochorismate lyase / aryl carrier protein [EC:3.3.2.1 6.3.2.14]',
  45],
 ['K01851  pchA; salicylate biosynthesis isochorismate synthase [EC:5.4.4.2]',
  79],
 ['K02361  entC; isochorismate synthase [EC:5.4.4.2]', 81],
 ['K02362  entD, pptT; enterobactin synthetase component D / holo-[acyl-carrier protein] synthase [EC:6.3.2.14 2.7.8.7]',
  4],
 ['K02363  entE, dhbE, vibE, mxcE; 2,3-dihydroxybenzoate---[aryl-carrier protein] ligase [EC:6.3.2.14 6.2.1.71]',
  1838],
 ['K02364  entF; L-serine---[L-seryl-carrier protein] ligase [EC:6.3.2.14 6.2.1.72]',
  2807],
 ['K02552  menF; menaquinone-specific isochorismate synthase [EC:5.4.4.2]',
  79],
 ['K04778  vibH; vibriobactin synthetase', 18],
 ['K04780  dhbF; glyine---[glycyl-carrier protein] ligase [EC:6.2.1.66]',
  2282],
 ['K04781  mbtI, irp9, ybtS; salicylate synthetase [EC:5.4.4.2 4.2.99.21]',
  73],

Save dicts to json

In [118]:
groups = ['transporters','synthesis']
dicts = [dict_transp_ko_contig, dict_synth_ko_contig]
sidero_ko_dir = 'siderophore_ko_contigs'
bn = os.path.splitext(fn)[0]
out_dir = workdir + '/' + sidero_ko_dir + '/' + bn 
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print("Made dir:", out_dir)

for group, d in zip(groups, dicts):
    out_fn = (
        out_dir + '/' 
        + bn + '.siderophore_' + group + '.dict_ko_contig.json'
    )
    with open(out_fn, 'w') as f:
        json.dump(d, f, sort_keys=True, indent=4, separators=(',', ': '))

### Search all gradients (non-selected)

Check dirs

In [123]:
gradients_dir = '/mnt/nfs/projects/armbrust-metat'
grad_kofam_dirnames = [
    'gradients3/g3_uw_pa_pm_metat/assemblies/annotations/kofam',
    'gradients3/g3_uw_pa_metat/assemblies/annotations/kofam',
    'gradients3/g3_diel_pa_metat/assemblies/annotations/kofam',
    'gradients3/g3_uw_ns_metat/assemblies/kofam',
    'gradients1/g1_station_ns_metat/assemblies/kofam',
    'gradients1/g1_station_pa_metat/assemblies/annotations/kofam',
    'gradients2/g2_dcm_rr_pa_metat/assemblies/kofam',
    'gradients2/g2_station_pa_metat/assemblies/annotations/kofam',
    'gradients2/g2_station_ns_metat/assemblies/kofam',
    'scope_diel/diel_pa_metat/assemblies/annotations/kofam',
]
for d in grad_kofam_dirnames:
    if not os.path.exists(gradients_dir + '/' + d):
        print(d)

Parse each file type

In [230]:
for d in grad_kofam_dirnames:
    fns = os.listdir(gradients_dir + '/' + d)
    for fn in fns:
        fn_full = gradients_dir + '/' + d + '/' + fn
        print(fn_full)
        fspl, ext = os.path.splitext(fn)
        ext2 = os.path.splitext(fspl)[1]
        if ext == '.csv' and not re.match(r"\w+_ALL$", fspl):
            # print(sum(1 for _ in open(fn_full)))
            with open(fn_full, 'r') as f:
                line = f.readline()
                l = line.split(',')
                print(l[1],l[4])
                line = f.readline()
                l = line.split(',')
                print(l[1],l[4])
        elif ext == '.gz' and ext2 == '.tsv':
            # print(sum(1 for _ in gzip.open(fn_full)))
            with gzip.open(fn_full, 'rt') as f:
                for _ in range(4):
                    line = f.readline()
                    l = re.split(r'\s', line)
                    l = list(filter(None, l))
                    print(l[0],l[1])




/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_pm_metat/assemblies/annotations/kofam/NPac.G3PA_PM.bf100.id99.aa.best.kofam.incT30.csv.gz
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_pm_metat/assemblies/annotations/kofam/NPac.G3PA_PM.bf100.id99.aa.tsv.gz
# gene
# ---------
G3.UW.PA.UW24_1.7m_PM.0.2um_TRINITY_DN224191_c0_g1_i1_6 K20651
G3.UW.PA.UW24_1.7m_PM.0.2um_TRINITY_DN223670_c0_g1_i2_3 K20762
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/zenodo_trimming
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/NPac.G3PA.kofam.tsv.gz
# gene
#----------------------------------------- ------
G3PA.UW39_TRINITY_DN1448753_c2_g1_i1_5 K06650
G3PA.UW39_TRINITY_DN1448753_c2_g1_i1_5 K12760
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/NPac.G3PA.UW.bf100.id99.aa.best.Kofam.incT30.csv.gz
/mnt/nfs/projects/armbrust-metat/gradients3/g3_diel_pa_metat/assemblies/annotations

Test speed

In [249]:
nreps = 1000

for d in grad_kofam_dirnames:
    fns = os.listdir(gradients_dir + '/' + d)
    for fn in fns:
        fn_full = gradients_dir + '/' + d + '/' + fn
        print(fn_full)
        fspl, ext = os.path.splitext(fn)
        ext2 = os.path.splitext(fspl)[1]

        dict_ko_contig = defaultdict(list)

        times = []
        i = 0
        if ext == '.csv' and not re.match(r"\w+_ALL$", fspl):
            with open(fn_full, 'r') as f:
                for line in f:
                    if i < nreps:
                        t0 = time()
                        l = line.split(',')
                        ko = l[4].replace('"','')
                        # print(ko, '\t', contig)
                        if ko in merged_KOs:
                            contig = l[1].replace('"','')
                            dict_ko_contig[ko].append(contig)
                        t1 = time()
                        times.append(t1-t0)
                        i += 1
                    else:
                        break
            print(np.mean(times))
        elif ext == '.gz' and ext2 == '.tsv':
            with gzip.open(fn_full, 'rt') as f:
                for line in f:
                    if i < nreps:
                        t0 = time()
                        l = re.split(r'\s', line)
                        l = list(filter(None, l))
                        ko = l[1]
                        # print(ko, '\t', contig)
                        if ko in merged_KOs:
                            dict_ko_contig[ko].append(l[0])
                        t1 = time()
                        times.append(t1-t0)
                        i += 1
                    else:
                        break
            print(np.mean(times))


/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_pm_metat/assemblies/annotations/kofam/NPac.G3PA_PM.bf100.id99.aa.best.kofam.incT30.csv.gz
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_pm_metat/assemblies/annotations/kofam/NPac.G3PA_PM.bf100.id99.aa.tsv.gz
3.2880306243896485e-06
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/zenodo_trimming
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/NPac.G3PA.kofam.tsv.gz
6.644725799560547e-06
/mnt/nfs/projects/armbrust-metat/gradients3/g3_uw_pa_metat/assemblies/annotations/kofam/NPac.G3PA.UW.bf100.id99.aa.best.Kofam.incT30.csv.gz
/mnt/nfs/projects/armbrust-metat/gradients3/g3_diel_pa_metat/assemblies/annotations/kofam/NPac.G3PA.diel.bf100.id99.aa.best.kofam.incT30.csv.gz
/mnt/nfs/projects/armbrust-metat/gradients3/g3_diel_pa_metat/assemblies/annotations/kofam/zenodo_trimming
/mnt/nfs/projects/armbrust-metat/gradients3/g3_diel_pa_metat/assemblies/annotation

In [259]:
'"abcd"'.replace('"','').replace("'", "") == 'abcd'

True

In [260]:
with open(fn_full, 'r') as f:
    print(type(f))


<class '_io.TextIOWrapper'>


In [261]:
os.path.split('abcd.ext')

('', 'abcd.ext')